Import library yang dibutuhkan

In [19]:
import duckdb

Siapkan DuckDB, httpfs (eksistensi duckdb yang bisa langsung query file by http), DuckLake (Eksistensi )

In [20]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

Minio Auth Setup

In [21]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

minio_host_only = minioAuth['endpoint'].replace('http://', '')

con.sql(f"SET s3_endpoint = '{minio_host_only}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")

# 📌 MATIKAN SSL dan gunakan STYLE PATH
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

Setup Source Path

In [22]:
# Path Metadata Catalog
catalogMetadata = './catalog/catalog.ducklake'

# Path Folder dimana file dikelola oleh catalog ducklake
managedStoredData = './managedStoreData/'

# Daftar file yang akan dikelola oleh catalog ducklake
minioObjectName = {
    'jakarta': 'staging/parquet/duckdb/JawaBarat/jakarta.parquet',
    'bekasi': 'staging/parquet/duckdb/JawaBarat/bekasi.parquet',
    'depok': 'staging/parquet/duckdb/JawaBarat/depok.parquet',
    'balikpapan': 'staging/parquet/duckdb/kalimantanTimur/balikpapan.parquet',
    'makassar': 'staging/parquet/duckdb/sulawesiSelatan/makassar.parquet',
    'palembang': 'staging/parquet/duckdb/sumateraSelatan/palembang.parquet'
}

Endpoint URL MinIO yang lengkap untuk target file yang akan dikelola

In [23]:
fullEndPointURL = {
    'jakarta': f"s3://{minioAuth['bucket']}/{minioObjectName['jakarta']}",
    'bekasi': f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}",
    'depok': f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}",
    'balikpapan': f"s3://{minioAuth['bucket']}/{minioObjectName['balikpapan']}",
    'makassar': f"s3://{minioAuth['bucket']}/{minioObjectName['makassar']}",
    'palembang': f"s3://{minioAuth['bucket']}/{minioObjectName['palembang']}"
}

In [24]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS LocalLake (DATA_PATH '{managedStoredData}');")
con.sql("USE LocalLake;")

con.sql("CREATE SCHEMA IF NOT EXISTS staging;")

In [25]:
loadJakarta = f"""
        CREATE OR REPLACE TABLE staging.dataJakarta AS 
        SELECT * FROM read_parquet('{fullEndPointURL['jakarta']}');
    """
loadBekasi = f"""
        CREATE OR REPLACE TABLE staging.dataBekasi AS 
        SELECT * FROM read_parquet('{fullEndPointURL['bekasi']}');
    """
loadDepok = f"""
        CREATE OR REPLACE TABLE staging.dataDepok AS 
        SELECT * FROM read_parquet('{fullEndPointURL['depok']}');
    """
loadBalikpapan = f"""
        CREATE OR REPLACE TABLE staging.dataBalikpapan AS 
        SELECT * FROM read_parquet('{fullEndPointURL['balikpapan']}');
    """
loadMakassar = f"""
        CREATE OR REPLACE TABLE staging.dataMakassar AS 
        SELECT * FROM read_parquet('{fullEndPointURL['makassar']}');
    """
loadPalembang = f"""
        CREATE OR REPLACE TABLE staging.dataPalembang AS 
        SELECT * FROM read_parquet('{fullEndPointURL['palembang']}');
    """

In [26]:
try:
    con.sql(loadJakarta)
    con.sql(loadBekasi)
    con.sql(loadDepok)
    con.sql(loadBalikpapan)
    con.sql(loadMakassar)
    con.sql(loadPalembang)
    print(f"✅ Semua tabel staging terdaftar. (Source: MinIO, Managed: Lokal)")

except Exception as e:
    print(f"❌ Gagal memuat tabel | Error: {e}")

✅ Semua tabel staging terdaftar. (Source: MinIO, Managed: Lokal)


In [29]:
print("\n--- Time Travel ke Data Depok (Versi 0) ---")
time_travel_query = f"""
    SELECT 
        *
    FROM 
        staging.dataJakarta;
"""
con.sql(time_travel_query).df()


--- Time Travel ke Data Depok (Versi 0) ---


,Nama_Sekolah,Provinsi,Kota,Kecamatan,NPSN,Status,Bentuk_Pendidikan,Status_Kepemilikan,SK_Pendirian_Sekolah,Tanggal_SK_Pendirian,...,Before_Ruang_TU,After_Ruang_TU,Before_Ruang_Konseling,After_Ruang_Konseling,Before_Ruang_OSIS,After_Ruang_OSIS,Before_Bangunan,After_Bangunan,Rombel,Kategorisasi_PD
0,SDN DUREN SAWIT 01,D.K.I. Jakarta,Kota Adm. Jakarta Timur,Kecamatan Duren Sawit,20108557,Negeri,SD,Pemerintah Daerah,859/196/I/AP/T/PEMDA/I/85,1910-01-01,...,0,0,0,0,0,0,1,1,18,Sedang
1,SDN DUREN SAWIT 05 PG,D.K.I. Jakarta,Kota Adm. Jakarta Timur,Kecamatan Duren Sawit,20108560,Negeri,SD,Pemerintah Daerah,No. 65/1985,1985-01-10,...,0,0,0,0,0,0,1,1,18,Sedang
2,SDN DUREN SAWIT 06,D.K.I. Jakarta,Kota Adm. Jakarta Timur,Kecamatan Duren Sawit,20108561,Negeri,SD,Pemerintah Pusat,None,1910-01-01,...,0,0,0,0,0,0,1,1,12,Sedang
3,SDN DUREN SAWIT 13 PG,D.K.I. Jakarta,Kota Adm. Jakarta Timur,Kecamatan Duren Sawit,20108568,Negeri,SD,Pemerintah Pusat,None,1910-01-01,...,0,0,0,0,0,0,1,1,16,Sedang
4,SDN DUREN SAWIT 15 PG,D.K.I. Jakarta,Kota Adm. Jakarta Timur,Kecamatan Duren Sawit,20108570,Negeri,SD,Pemerintah Daerah,2452,1985-12-06,...,0,0,0,0,0,0,1,1,11,Sedang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,SMPI AL JIHAD,D.K.I. Jakarta,Kota Adm. Jakarta Pusat,Kecamatan Johar Baru,20106360,Swasta,SMP,Yayasan,418.p/101.A1/90,1990-02-12,...,1,1,1,1,1,1,1,1,11,Sedang
1749,SMP PASKALIS 1,D.K.I. Jakarta,Kota Adm. Jakarta Pusat,Kecamatan Johar Baru,20106426,Swasta,SMP,Yayasan,SP.477/101.1A/85,1985-12-16,...,1,1,2,2,1,1,1,1,3,Sedikit
1750,SMP YWPM,D.K.I. Jakarta,Kota Adm. Jakarta Pusat,Kecamatan Johar Baru,20106442,Swasta,SMP,Yayasan,AKTE NO.76 TAHUN 1989,1976-07-01,...,1,1,3,3,2,2,1,1,5,Sedikit
1751,SMKN 14 JAKARTA,D.K.I. Jakarta,Kota Adm. Jakarta Pusat,Kecamatan Johar Baru,20100158,Negeri,SMK,Pemerintah Pusat,D/IV/4991/II/76,1900-01-01,...,1,1,1,1,1,1,1,1,26,Banyak


In [30]:
con.close()